# CSCE 5380 Spring 2018 - Data Mining - Lab Assignment 1 - Task 3

Constant Marks

<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Image-Segmentation-Data" data-toc-modified-id="Image-Segmentation-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Image Segmentation Data</a></span></li><li><span><a href="#Large-Soybean-Database" data-toc-modified-id="Large-Soybean-Database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Large Soybean Database</a></span></li><li><span><a href="#Thyroid-Disease-Records" data-toc-modified-id="Thyroid-Disease-Records-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Thyroid Disease Records</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Problem-5:-Error-rate-and-accuracy-for-each-of-the-datasets" data-toc-modified-id="Problem-5:-Error-rate-and-accuracy-for-each-of-the-datasets-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Problem 5: Error-rate and accuracy for each of the datasets</a></span></li></ul></li></ul></li></ul></div>

Run the Decision Tree classifier on [Image Segmentation data](data/segment.arff), [Large Soybean Database](data/soybean.arff), and [Thyroid disease records](data/sick.arff) with a 72% and 28% split of training and test data respectively.

## Image Segmentation Data

Import the image segment dataset, display the first record, and the number of records. 

In [1]:
from scipy.io.arff import loadarff
with open("data/segment.arff", "r") as f:
    data, meta = loadarff(f)
print(data[0],'\n')
print('There are %d records. ' % (data.size))



( 218.,  178.,  9.,  0.111111,  0.,  0.833333,  0.547722,  1.11111,  0.544331,  59.6296,  52.4444,  75.2222,  51.2222, -21.5556,  46.7778, -25.2222,  75.2222,  0.318996, -2.04055, b'path') 

There are 2310 records. 


Slice out the feature set and save as numpy array. 

In [2]:
X = data[meta.names()[0:-1]]
import numpy as np
X = np.asarray(X.tolist())
print(X[0])

[  2.18000000e+02   1.78000000e+02   9.00000000e+00   1.11111000e-01
   0.00000000e+00   8.33333000e-01   5.47722000e-01   1.11111000e+00
   5.44331000e-01   5.96296000e+01   5.24444000e+01   7.52222000e+01
   5.12222000e+01  -2.15556000e+01   4.67778000e+01  -2.52222000e+01
   7.52222000e+01   3.18996000e-01  -2.04055000e+00]


Slice out the class data, save as numpy array, identify the unique classes, and display the number of classes. 

In [3]:
y = data[meta.names()[-1]]
y = np.asarray(y.tolist())
print(np.unique(y))
print('\nThere are %d classes.' %(np.size(np.unique(y))))


[b'brickface' b'cement' b'foliage' b'grass' b'path' b'sky' b'window']

There are 7 classes.


Split the dataset into test and training sets.

In [4]:
# Splitting data into test and training set choosing a 10% sample Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.28)

Create and train the model, count the nodes, display the tree with graphviz, create the confusion matrix, and use k-fold cross validation to calculate the accuracy and error rates.  

In [5]:
# Building the Model
from sklearn.tree import DecisionTreeClassifier, export_graphviz
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)
leaves = classifier.tree_.node_count
export_graphviz(classifier, out_file = 'graphs/segment_tree.dot')

# Viewing the Tree 
import graphviz 
graphviz.view('graphs/segment_tree.dot')

# Predictinging the Test set results
y_pred = classifier.predict(X_test)

# Evaluating the Accuracy 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn.model_selection import cross_val_score
scores= cross_val_score(classifier, X_train, y_train, cv=5)
errors = 1-scores
print(cm)
print("\nDecision Tree (%d leaves) Accuracy on Segment Data: %0.2f (+/- %0.2f)" % (leaves, scores.mean(), scores.std() * 2))
print("Decision Tree (%d leaves) Error on Segment Data: %0.2f (+/- %0.2f)\n" % (leaves, errors.mean(), errors.std() * 2))

[[ 82   1   1   0   0   0   0]
 [  0  80   0   0   1   0   3]
 [  1   1  79   0   0   0   2]
 [  0   0   0 103   0   0   1]
 [  0   1   0   0  79   0   0]
 [  0   0   0   0   0 102   0]
 [  0   1   9   0   0   0 100]]

Decision Tree (127 leaves) Accuracy on Segment Data: 0.95 (+/- 0.02)
Decision Tree (127 leaves) Error on Segment Data: 0.05 (+/- 0.02)



## Large Soybean Database

Import the image segment dataset, display the first record, and the number of records. 

In [6]:
from scipy.io.arff import loadarff
with open("data/soybean.arff", "r") as f:
    data, meta = loadarff(f)
print(data[0],'\n')
print('There are %d records: ' % (data.size))

(b'october', b'normal', b'gt-norm', b'norm', b'yes', b'same-lst-yr', b'low-areas', b'pot-severe', b'none', b'90-100', b'abnorm', b'abnorm', b'absent', b'dna', b'dna', b'absent', b'absent', b'absent', b'abnorm', b'no', b'above-sec-nde', b'brown', b'present', b'firm-and-dry', b'absent', b'none', b'absent', b'norm', b'dna', b'norm', b'absent', b'absent', b'norm', b'absent', b'norm', b'diaporthe-stem-canker') 

There are 683 records: 


Slice out the feature set, save as numpy array, and encode the features. 

In [7]:
X = data[meta.names()[0:-1]]
import numpy as np
X = np.asarray(X.tolist())
print(X[0])
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
for i in range(np.shape(X)[1]):
    X[:,i] = labelencoder_X.fit_transform(X[:, i])
X = X.astype(int)
print(X[0:5],'\n')

[b'october' b'normal' b'gt-norm' b'norm' b'yes' b'same-lst-yr' b'low-areas'
 b'pot-severe' b'none' b'90-100' b'abnorm' b'abnorm' b'absent' b'dna'
 b'dna' b'absent' b'absent' b'absent' b'abnorm' b'no' b'above-sec-nde'
 b'brown' b'present' b'firm-and-dry' b'absent' b'none' b'absent' b'norm'
 b'dna' b'norm' b'absent' b'absent' b'norm' b'absent' b'norm']
[[6 2 1 3 2 4 1 2 2 1 1 0 1 1 1 1 1 1 1 1 1 1 2 2 1 3 1 4 4 2 1 1 2 1 2]
 [2 2 1 3 2 3 2 3 1 0 1 0 1 1 1 1 1 1 1 2 1 1 2 2 1 3 1 4 4 2 1 1 2 1 2]
 [3 2 1 3 2 4 2 3 1 3 1 0 1 1 1 1 1 1 1 2 1 3 2 2 1 3 1 4 4 2 1 1 2 1 2]
 [3 2 1 3 2 4 2 3 2 0 1 0 1 1 1 1 1 1 1 2 1 3 2 2 1 3 1 4 4 2 1 1 2 1 2]
 [6 2 1 3 2 3 2 2 2 3 1 0 1 1 1 1 1 1 1 2 1 1 2 2 1 3 1 4 4 2 1 1 2 1 2]] 



Slice out the class data, save as numpy array, identify the unique classes, and display the number of classes. 

In [8]:
y = data[meta.names()[-1]]
y = np.asarray(y.tolist())
print(np.unique(y))
print('\nThere are %d classes' %(np.size(np.unique(y))))

[b'2-4-d-injury' b'alternarialeaf-spot' b'anthracnose' b'bacterial-blight'
 b'bacterial-pustule' b'brown-spot' b'brown-stem-rot' b'charcoal-rot'
 b'cyst-nematode' b'diaporthe-pod-&-stem-blight' b'diaporthe-stem-canker'
 b'downy-mildew' b'frog-eye-leaf-spot' b'herbicide-injury'
 b'phyllosticta-leaf-spot' b'phytophthora-rot' b'powdery-mildew'
 b'purple-seed-stain' b'rhizoctonia-root-rot']

There are 19 classes


Split the dataset into test and training sets.

In [9]:
# Splitting data into test and training set choosing a 10% sample Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.28)

Create and train the model, count the nodes, display the tree with graphviz, create the confusion matrix, and use k-fold cross validation to calculate the accuracy and error rates.  

In [10]:
# Building the Model
from sklearn.tree import DecisionTreeClassifier, export_graphviz
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)
leaves = classifier.tree_.node_count
export_graphviz(classifier, out_file = 'graphs/soybean_tree.dot')

# Viewing the Tree 
import graphviz 
graphviz.view('graphs/soybean_tree.dot')

# Predictinging the Test set results
y_pred = classifier.predict(X_test)

# Evaluating the Accuracy 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn.model_selection import cross_val_score
scores= cross_val_score(classifier, X_train, y_train, cv=5)
print(cm)
print("\nDecision Tree (%d leaves) Accuracy on Soybean Data: %0.2f (+/- %0.2f)" % (leaves, scores.mean(), scores.std() * 2))
print("Decision Tree (%d leaves) Error on Soybean Data: %0.2f (+/- %0.2f)\n" % (leaves, errors.mean(), errors.std() * 2))

[[ 6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 30  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0  0  0  1  0  1  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  1  0  0  0  0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  1  0  0  0  1  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 25  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0

## Thyroid Disease Records

In [11]:
from scipy.io.arff import loadarff
with open("data/sick.arff", "r") as f:
    data, meta = loadarff(f)
print(data[0],'\n')
print('There are %d records: ' % (data.size))

( 41., b'F', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b'f', b't',  1.3, b't',  2.5, b't',  125., b't',  1.14, b't',  109., b'f',  nan, b'SVHC', b'negative') 

There are 3772 records: 


Slice out the feature set, save as numpy array, and encode the data. 

In [12]:
X = data[meta.names()[0:-1]]
import numpy as np
X = np.asarray(X.tolist())
print(X[0])
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
for i in range(np.shape(X)[1]):
    X[:,i] = labelencoder_X.fit_transform(X[:, i])
X = X.astype(int)
print(X[0:5],'\n')

[b'41.0' b'F' b'f' b'f' b'f' b'f' b'f' b'f' b'f' b'f' b'f' b'f' b'f' b'f'
 b'f' b'f' b't' b'1.3' b't' b'2.5' b't' b'125.0' b't' b'1.14' b't' b'109.0'
 b'f' b'nan' b'SVHC']
[[ 34   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1 110
    1  27   1  28   1  72   1  10   0   0   1]
 [ 15   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1 195
    1  22   1   3   0 146   0 234   0   0   4]
 [ 40   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1 103
    0  69   1  10   1  48   1  22   0   0   4]
 [ 67   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   1  21
    1  20   1  83   0 146   0 234   0   0   4]
 [ 67   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1  77
    1  12   1 201   1  44   1 199   0   0   3]] 



Slice out the class data, save as numpy array, identify the unique classes, and display the number of classes. 

In [13]:
y = data[meta.names()[-1]]
y = np.asarray(y.tolist())
print(np.unique(y))
print('\nThere are %d classes' %(np.size(np.unique(y))))



[b'negative' b'sick']

There are 2 classes


Split the dataset into test and training sets.

In [14]:
# Splitting data into test and training set choosing a 10% sample Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.28)

Create and train the model, count the nodes, display the tree with graphviz, create the confusion matrix, and use k-fold cross validation to calculate the accuracy and error rates.  

In [15]:
# Building the Model
from sklearn.tree import DecisionTreeClassifier, export_graphviz
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)
leaves = classifier.tree_.node_count
export_graphviz(classifier, out_file = 'graphs/sick_tree.dot')

# Viewing the Tree 
import graphviz 
graphviz.view('graphs/sick_tree.dot')

# Predictinging the Test set results
y_pred = classifier.predict(X_test)

# Evaluating the Accuracy 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn.model_selection import cross_val_score
scores= cross_val_score(classifier, X_train, y_train, cv=5)
print(cm)
print("\nDecision Tree (%d leaves) Accuracy on Thyroid Disease Data: %0.2f (+/- %0.2f)\n" % (leaves, scores.mean(), scores.std() * 2))
print("Decision Tree (%d leaves) Error on Thyroid Disease Data: %0.2f (+/- %0.2f)\n" % (leaves, errors.mean(), errors.std() * 2))

[[984  11]
 [  9  53]]

Decision Tree (127 leaves) Accuracy on Thyroid Disease Data: 0.98 (+/- 0.00)

Decision Tree (127 leaves) Error on Thyroid Disease Data: 0.05 (+/- 0.02)



#### Problem 5: Error-rate and accuracy for each of the datasets

Calculate and report the error-rate and accuracy for each of the datasets. Which of the 3 datasets has the highest number of correctly classified instances?

   **Answer:** 
   
    Decision Tree (125 leaves) Accuracy Rate on Segment Data: 0.96 (+/- 0.02)
    Decision Tree (125 leaves) Error Rate on Segment Data: 0.04 (+/- 0.02)

    Decision Tree (131 leaves) Accuracy Rate on Soybean Data: 0.92 (+/- 0.07)
    Decision Tree (131 leaves) Error Rate on Soybean Data: 0.04 (+/- 0.02)

    Decision Tree (103 leaves) Accuracy on Thyroid Disease Data: 0.98 (+/- 0.01)
    Decision Tree (103 leaves) Error on Thyroid Disease Data: 0.04 (+/- 0.02)
    
    The Thyroid Disease dateset has the highest accuracy rate. 



Which of the three datasets has the smallest and largest decision trees? Explain why you think the size of the decision trees varies.

   **Answer**
   
   The size of the decision tree is proportional to the number of classes. The soybean dataset had the largest decision tree(131 nodes/leaves) as well as the largest number of classes (19).  The image segment dataset had the next largest tree(125 nodes/leaves) and 7 classes.  The Thyroid disease set had a binary class and the smallest tree size(103 nodes/leaves).